In [ ]:
import pandas as pd
import numpy as np
import scipy.io as sio
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import KFold
import graphviz

In [ ]:
# Carregando o arquivo
dados = sio.loadmat('/content/Dataset.mat')

In [ ]:
 # Extraindo os dados
dados = dados['Dataset']

In [ ]:
# Convertendo o array em um DataFrame
dados = pd.DataFrame(dados)

In [ ]:
# Visualizando os dados
dados

,0,1,2,3
0,1,0,0,1
1,0,0,0,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,1
...,...,...,...,...
871,1,1,0,1
872,1,1,0,1
873,1,1,0,1
874,1,0,1,1


In [ ]:
# Renomeando colunas
dados = dados.rename(columns={0: 'Empregado', 1: 'Devedor', 2: 'Salário acima de 5SM', 3: 'Empréstimo'})

In [ ]:
# Visualizando os dados, após renomeação.
dados

,Empregado,Devedor,Salário acima de 5SM,Empréstimo
0,1,0,0,1
1,0,0,0,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,1
...,...,...,...,...
871,1,1,0,1
872,1,1,0,1
873,1,1,0,1
874,1,0,1,1


In [ ]:
# Mostrando as informações dos dados
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Empregado             876 non-null    uint8
 1   Devedor               876 non-null    uint8
 2   Salário acima de 5SM  876 non-null    uint8
 3   Empréstimo            876 non-null    uint8
dtypes: uint8(4)
memory usage: 3.5 KB


In [ ]:
# Mostrando estatísticas básicas dos dados
dados.describe()

,Empregado,Devedor,Salário acima de 5SM,Empréstimo
count,876.000000,876.000000,876.000000,876.000000
mean,0.469178,0.493151,0.542237,0.472603
std,0.499334,0.500239,0.498497,0.499534
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Mostrando todos os dados individualmente
for i in dados.columns:
  print(i)
  print(dados[i].unique())
  print('\n')

Empregado
[1 0]


Devedor
[0 1]


Salário acima de 5SM
[0 1]


Empréstimo
[1 0]




In [ ]:
# Contando a quantidade de valores em cada coluna
cont_1 = dados.apply(lambda x: (x == 1).sum())
cont_0 = dados.apply(lambda x: (x == 0).sum())

# Criando um novo DataFrame com os resultados
resul_cont = pd.DataFrame({
    'Quantidade de 1': cont_1,
    'Quantidade de 0': cont_0
})

Podemos ver que existe um número parecido de resultados nos dados, sendo assim não tem a necessidade de um balanciamento.

In [ ]:
# Mostrando quantidade de valores em cada coluna
resul_cont

,Quantidade de 1,Quantidade de 0
Empregado,411,465
Devedor,432,444
Salário acima de 5SM,475,401
Empréstimo,414,462


Extrai os rótulos do conjunto de dados e calcula as frequências de cada rótulo. Com essas frequências, determina as probabilidades das classes e utiliza essas probabilidades para calcular a entropia, que mede a incerteza ou desordem dos dados.

In [ ]:
# Calculando a entropia dos rótulos no conjunto de dados
def calcular_entropia(dados):
    rotulos = dados[:, -1]
    total_instancias = len(rotulos)
    _, contagens = np.unique(rotulos, return_counts=True)
    probabilidades = contagens / total_instancias
    entropia = -np.sum(probabilidades * np.log2(probabilidades + 1e-9))

    return entropia

Calcula a entropia total dos dados e, em seguida, calcula a entropia dos subconjuntos resultantes da divisão para cada valor do atributo. Dessa forma, a entropia ponderada é então calculada usando as proporções dos subconjuntos. Finalmente, a diferença entre a entropia total e a entropia ponderada é chamada de ganho de informação, indicando que a divisão diminuiu a incerteza.

In [ ]:
# Calculando o ganho de informação para um determinado atributo
def calcular_ganho_informacao(dados, indice_atributo):
    entropia_total = calcular_entropia(dados)
    valores, contagens = np.unique(dados[:, indice_atributo], return_counts=True)
    entropia_ponderada = 0
    total_instancias = len(dados)
    for valor, contagem in zip(valores, contagens):
        subconjunto = dados[dados[:, indice_atributo] == valor]
        entropia_subconjunto = calcular_entropia(subconjunto)
        entropia_ponderada += (contagem / total_instancias) * entropia_subconjunto

    return entropia_total - entropia_ponderada

Determina o melhor atributo para dividir os dados calculando o ganho de informação de cada um e seleciona o que oferece a maior redução na incerteza.

In [ ]:
# Encontrando o melhor atributo para dividir os dados
def melhor_divisao(dados, atributos):
    melhor_ganho = -1
    melhor_atributo = None
    for atributo in atributos:
        ganho = calcular_ganho_informacao(dados, atributo)
        if ganho > melhor_ganho:
            melhor_ganho = ganho
            melhor_atributo = atributo

    return melhor_atributo

Constrói a árvore de decisão recursivamente. Verifica se todas as instâncias têm o mesmo rótulo ou se a profundidade máxima foi atingida, retornando o rótulo mais frequente se necessário. Caso contrário, seleciona o melhor atributo para divisão e cria um nó interno, chamando a função recursivamente para cada valor do atributo até que todos os subconjuntos sejam processados.

In [ ]:
# Construindo uma árvore de decisão de forma recursiva
def construir_arvore(dados, atributos, profundidade_maxima=None, profundidade=0):
    rotulos = dados[:, -1]

    # Verificando as condições de parada para a construção da árvore de decisão
    if len(np.unique(rotulos)) == 1:
        return rotulos[0]

    if len(atributos) == 0 or (profundidade_maxima is not None and profundidade >= profundidade_maxima):
        return np.bincount(rotulos.astype(int)).argmax()

    melhor_atributo = melhor_divisao(dados, atributos)
    if melhor_atributo is None:
        return np.bincount(rotulos.astype(int)).argmax()

    # Construindo a árvore de decisão
    arvore = {"atributo": melhor_atributo, "nos": {}}
    valores = np.unique(dados[:, melhor_atributo])
    novos_atributos = [attr for attr in atributos if attr != melhor_atributo]

    for valor in valores:
        subconjunto = dados[dados[:, melhor_atributo] == valor]
        arvore["nos"][valor] = construir_arvore(subconjunto, novos_atributos, profundidade_maxima, profundidade + 1)

    return arvore

Faz previsões com a árvore de decisão seguindo o caminho baseado no atributo de divisão. Se o valor da instância não estiver presente, retorna o rótulo majoritário. Se o nó for uma folha, retorna o rótulo final.

In [ ]:
# Fazendo as previsões introduzida na árvore de decisão
def prever(arvore, instancia, rotulo_majoritario):
    if isinstance(arvore, dict):
        atributo = arvore["atributo"]
        valor = instancia[atributo]
        if valor in arvore["nos"]:
            return prever(arvore["nos"][valor], instancia, rotulo_majoritario)
        else:
            return rotulo_majoritario
    else:
        return arvore

Avalia o desempenho do modelo calculando acurácia, sensibilidade, especificidade, precisão e F1-score, fornecendo uma visão geral do seu desempenho.

In [ ]:
# Avaliando o modelo usando as métricas (acurácia, sensibilidade, especificidade, precisão e F1-score)
def avaliar_metricas(rotulos, previsoes):
    acuracia = accuracy_score(rotulos, previsoes)
    sensibilidade = recall_score(rotulos, previsoes, pos_label=1, zero_division=0)
    especificidade = recall_score(rotulos, previsoes, pos_label=0, zero_division=0)
    precisao = precision_score(rotulos, previsoes, pos_label=1, zero_division=0)
    f1 = f1_score(rotulos, previsoes, pos_label=1, zero_division=0)

    return acuracia, sensibilidade, especificidade, precisao, f1

Realiza a validação cruzada dividindo os dados em
k partes. Para cada iteração, treina o modelo com
k - 1 partes e testa com a parte restante. Em seguida, calcula as métricas de desempenho (acurácia, sensibilidade, especificidade, precisão e F1-score) e calcula a média dessas métricas ao final para avaliar o modelo.

In [ ]:
# Executando a validação cruzada para avaliar o modelo
def validacao_cruzada(dados, k=10):
    n_amostras = len(dados)
    k = min(k, n_amostras)
    kf = KFold(n_splits=k)
    acuracias, sensibilidades, especificidades, precisoes, f1s = [], [], [], [], []

    # Loop sobre os índices de treinamento e teste gerados pela validação cruzada.
    for indice_treino, indice_teste in kf.split(dados):
        dados_treino, dados_teste = dados[indice_treino], dados[indice_teste]
        atributos = list(range(dados_treino.shape[1] - 1))
        rotulo_majoritario = np.bincount(dados_treino[:, -1].astype(int)).argmax()
        arvore = construir_arvore(dados_treino, atributos, profundidade_maxima=len(atributos))

        previsoes_teste = [prever(arvore, instancia[:-1], rotulo_majoritario) for instancia in dados_teste]
        rotulos_teste = dados_teste[:, -1]
        metricas = avaliar_metricas(rotulos_teste, previsoes_teste)

        acuracias.append(metricas[0])
        sensibilidades.append(metricas[1])
        especificidades.append(metricas[2])
        precisoes.append(metricas[3])
        f1s.append(metricas[4])

    # Cálculando as médias das métricas ao longo de todas as iterações.
    media_acuracia = np.mean(acuracias)
    media_sensibilidade = np.mean(sensibilidades)
    media_especificidade = np.mean(especificidades)
    media_precisao = np.mean(precisoes)

    return media_acuracia, media_sensibilidade, media_especificidade, media_precisao, np.mean(f1s)

In [ ]:
# Carregando os dados
dados = np.array(dados)

# Executando validação cruzada
media_acuracia, media_sensibilidade, media_especificidade, media_precisao, media_f1 = validacao_cruzada(dados, k=10)

In [ ]:
# Exibindo os resultados
print(f"Acurácia média: {media_acuracia:.2f}")
print(f"Sensibilidade média: {media_sensibilidade:.2f}")
print(f"Especificidade média: {media_especificidade:.2f}")
print(f"Precisão média: {media_precisao:.2f}")
print(f"F1-score médio: {media_f1:.2f}")

Acurácia média: 0.89
Sensibilidade média: 0.89
Especificidade média: 0.90
Precisão média: 0.89
F1-score médio: 0.89
